In [1]:
import ROOT
import math
from array import array
import numpy as np

Welcome to JupyROOT 6.26/00


In [2]:
ls -l *.root

-rw-r--r--  1 akhanal  staff       44490 Feb 13  2022 basic.root
-rw-r--r--  1 akhanal  staff      890817 Feb 22  2022 data_all_fde.root
-rw-r--r--  1 akhanal  staff     7487446 Feb 22  2022 data_all_fte.root
-rw-r--r--  1 akhanal  staff     7391613 Feb 22  2022 data_all_fte_ecornonly.root
-rw-r--r--  1 akhanal  staff     7233824 Feb 22  2022 data_all_fte_nocorn.root
-rw-r--r--  1 akhanal  staff      243193 Feb 21  2022 data_fall2018_in_fde.root
-rw-r--r--  1 akhanal  staff      256433 Jun 16 22:29 data_fall2018_in_fde_goodruns.root
-rw-r--r--  1 akhanal  staff      273170 Jul 28 18:57 data_fall2018_in_fde_goodruns_hasLamSigWeight.root
-rw-r--r--  1 akhanal  staff   202574473 Jul 28 16:37 data_fall2018_in_fde_goodruns_threeKaon_mixEv_bg.root
-rw-r--r--  1 akhanal  staff     3130991 Feb 22  2022 data_fall2018_in_fte.root
-rw-r--r--  1 akhanal  staff     3090975 Feb 22  2022 data_fall2018_in_fte_ecornonly.root
-rw-r--r--  1 akhanal  staff     3142907 Jun 16 22:51 data_fall2018_in_fte_goo

In [3]:
df = ROOT.RDataFrame("T", "data_xi1320_2kplus_fall2018_out_fde.root")
print(f"Columns in the dataset: {df.GetColumnNames()}")

Columns in the dataset: { "W", "beam_e", "e_e", "e_p", "e_phi", "e_px", "e_py", "e_pz", "e_theta", "e_vx", "e_vy", "e_vz", "evnum", "fkp_e", "fkp_p", "fkp_phi", "fkp_px", "fkp_py", "fkp_pz", "fkp_theta", "fkp_vx", "fkp_vy", "fkp_vz", "m_fkpskp", "mm_e", "mm_fkp", "mm_fkpskp", "mm_skp", "nu", "q2", "runnum", "skp_e", "skp_p", "skp_phi", "skp_px", "skp_py", "skp_pz", "skp_theta", "skp_vx", "skp_vy", "skp_vz", "x" }


In [4]:
%%cpp 
double background(Double_t *x, Double_t *par) {
  return par[0] + par[1]*x[0];
}

In [5]:
%%cpp
Double_t p2background(Double_t *x, Double_t *par) {
  //return par[0] + par[1]*x[0] + par[2]*x[0]*x[0]+ par[3]*x[0]*x[0]*x[0];
  return par[0] + par[1]*x[0] + par[2]*x[0]*x[0];
}

In [6]:
%%cpp
Double_t p3background(Double_t *x, Double_t *par) {
  return par[0] + par[1]*x[0] + par[2]*x[0]*x[0]+ par[3]*x[0]*x[0]*x[0];
  
}

In [7]:
%%cpp 
double backgroundPoly(Double_t *x) {
  return 2*x[0]*x[0]-1;
}

In [8]:
%%cpp
Double_t Gaussian(Double_t *x, Double_t *par)
{
    Double_t arg = 0;
    if (par[2]) arg = (x[0] - par[1])/par[2];
    Double_t sig = par[0]*TMath::Exp(-0.5*arg*arg);
    return sig;
}

In [9]:
%%cpp
Double_t GaussianPeak(Double_t *x, Double_t *par)
{
    Double_t arg = 0;
    if (par[2]) arg = (x[0] - par[1])/par[2];
    Double_t sig = par[0]*TMath::Exp(-0.5*arg*arg);
    return sig;
}

In [10]:
%%cpp
double p2fitDoubleGaus(double *x, double *par) {
  return GaussianPeak(x, par) + Gaussian(x, &par[3]) + p2background(x, &par[6]);
}

In [11]:
%%cpp
double fitDoubleGaus(double *x, double *par) {
  return GaussianPeak(x, par) + Gaussian(x, &par[3]) + background(x, &par[6]);
}

In [12]:
%%cpp
double fitGaus(double *x, double *par) {
    return GaussianPeak(x, par) + background(x, &par[3]);
    //return GaussianPeak(x, par) + p2background(x, &par[3]);
}

In [13]:
%%cpp
double fitp3Gaus(double *x, double *par) {
  return GaussianPeak(x, par) + p3background(x, &par[3]);
    //+ par[3]+par[4]*x[0]+par[5]*x[0]*x[0]+par[6]*x[0]*x[0]*x[0];
}

In [14]:
%%cpp
Double_t lorentzianPeak(Double_t *x, Double_t *par) {
    return (0.5*par[0]*par[2]/TMath::Pi()) /
    TMath::Max( 1.e-10,
               (x[0]-par[1])*(x[0]-par[1]) + .25*par[2]*par[2]
               );
}

In [15]:
%%cpp
Double_t lorentzian(Double_t *x, Double_t *par) {
    return (0.5*par[0]*par[2]/TMath::Pi()) /
    TMath::Max( 1.e-10,
               (x[0]-par[1])*(x[0]-par[1]) + .25*par[2]*par[2]
               );
}

In [16]:
%%cpp
Double_t fitLorentzianPeak(Double_t *x, Double_t *par) {
    return lorentzianPeak(x, par) + p2background(x, &par[3]);
    //return lorentzianPeak(x, par) + backgroundPoly(x);
}

In [17]:
%%cpp
double fitBW(double *x, double *par) {
  return lorentzianPeak(x, par) + background(x, &par[3]);
}

In [18]:
%%cpp
double p2fitDoubleBW(Double_t *x, Double_t *par){
    return lorentzianPeak(x, par) + lorentzian(x, &par[3]) + p2background(x, &par[6]);
}

In [19]:
%%cpp
float low_3sigma(TF1 *fcn){
    return fcn->GetParameter(1)-3*fcn->GetParameter(2);
}

In [20]:
%%cpp
float high_3sigma(TF1 *fcn){
    return fcn->GetParameter(4)+3*fcn->GetParameter(5);
}

In [21]:
%%cpp
bool sigmaCut(TF1 *fcn){
    if(fcn->GetParameter(1)-3*fcn->GetParameter(2) && fcn->GetParameter(4)+3*fcn->GetParameter(5)){
        return true;
    }
    else {
        return false;
    }  
}

In [22]:
%%cpp
Double_t Voigt(Double_t *x, Double_t *par) {
    Double_t arg = 0;
    if (par[0]) arg = (x[0] - par[0]);
    return TMath::Voigt(arg, par[1], par[2], 4);
    // median, sigma, lg
}

**Xi(1320) with FDe - mc - gaussian fit**

In [23]:
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p1p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p2p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p3p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p35p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p4p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p5p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p6p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p65p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p7p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p8p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p9p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p2p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p4p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")

h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(1)
c = ROOT.TCanvas()
h.Draw("E")
h.Fit("gaus", "S", " ", 1.28, 1.38)
#h.Fit("gaus", "S", " ", 1.26, 1.4)#0p6p
#h.Fit("gaus", "S", " ", 1.24, 1.42)#0p8p
#h.Fit("gaus", "S", " ", 1.2, 1.48)#1p2p
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
c.Draw()

 FCN=52.6698 FROM MIGRAD    STATUS=CONVERGED      72 CALLS          73 TOTAL
                     EDM=4.76817e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     9.66429e+03   7.57549e+01   2.01229e-01   1.54662e-06
   2  Mean         1.32941e+00   1.69685e-04   6.33914e-07  -2.58504e-01
   3  Sigma        2.54910e-02   1.74398e-04   6.51833e-06   3.25078e-02


In [24]:
p_smear = array('d', [0, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.65, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4])
err_p_smear = array('d', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#chi2 = [5.40, 5.42, 5.11, 5.07, 4.84, 4.63, 4.23, 3.49, 3.42, 2.65, 1.84, 1.69]
#sig_xi1320 = array('d', [0.0337,0.0343, 0.0373, 0.0416, 0.0452, 0.0473, 0.0517, 0.0577, 0.0614, 0.0639, 0.0700, 0.0761, 0.0837, 0.1002, 0.1163])
sig_xi1320 = array('d', [0.02065,0.02089, 0.02223, 0.02426, 0.02549, 0.02699, 0.03195, 0.03479, 0.03847, 0.03988, 0.04292, 0.04954, 0.05314, 0.06032, 0.06681])
#err_sig_xi1320 = array('d', [0.0002,0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0004, 0.0004, 0.0005, 0.0005, 0.0005, 0.0006, 0.0006, 0.0008, 0.0009])
err_sig_xi1320 = array('d', [0.00013,0.00013, 0.00014, 0.00016, 0.00017, 0.00020, 0.00019, 0.00023, 0.00021, 0.00022, 0.00026, 0.00025, 0.00027, 0.00032, 0.00039])
gr = ROOT.TGraphErrors(len(p_smear), p_smear, sig_xi1320, err_p_smear, err_sig_xi1320)
#gr = ROOT.TGraph(len(p_smear), p_smear, sig_lam)
gr.SetTitle("#Xi^{-}(1320) width in MM(eK^{+}K^{+}) from MC (smeared))")
#gr.SetTitle("#sigma(of #Xi^{-}(1320)fit from MC)")
gr.GetXaxis().SetTitle("Gaussian smearing factor FD-particle #Delta p/p(%)")
gr.GetYaxis().SetTitle("#sigma [GeV/c^{2}]")
gr.SetMarkerStyle(21)
gr.SetMarkerSize(0.8)
gr.SetLineColor(ROOT.kRed)
gr.SetMarkerColor(ROOT.kGreen)
gr.SetFillStyle(1001)

%jsroot on
ROOT.gStyle.SetOptTitle(1)
c = ROOT.TCanvas()
gr.Draw("AC*")

#hbox = ROOT.TH2F("hbox(name)","Option BOX example(title)",3(nbinsx),0(xlow),3(xup),3(nbinsy),0(ylow),3(yup))# did not work
#band = ROOT.TH2F("band", "",1 , c.GetUxmin(), gr.GetXaxis().GetXmax(), 1, 0.465, 0.0565)
#band.SetFillColor(42)
#band.Fill(1, 0.0515, 0.01)
#band.Draw("SAME")
"""
multigraph = ROOT.TMultiGraph()
# lower bound
x1 = array('d', [0,1])
y1 = array('d', [0.465,0.465])
excl1 = ROOT.TGraph(5,x1,y1)
excl1.SetLineColor(41)
excl1.SetFillColor(41)
excl1.SetFillStyle(1001)
# upper bound
x1 = array('d', [0,1])
y2 = array('d', [0.0565,0.0565])
excl2 = ROOT.TGraph(5,x1,y2)
excl2.SetLineColor(46)
excl2.SetFillColor(46)
#excl2.SetFillStyle(1001)

multigraph.Add(excl1,"LF")
multigraph.Add(excl2,"LF")
#multigraph.Draw("same")
"""

t1 = ROOT.TLatex(0.8, 0.03, "#Xi^{-}(1320) width from data")
t1.Draw("SAME")

#pt = ROOT.TPaveText(0,0.0465,1.4,0.0565)
#pt.AddText("")
#pt.SetTextColor(1)
#pt.SetTextSize(0.04)
#pt.SetShadowColor(ROOT.kGray)
#pt = ROOT.TPave(0,0.0465,1.4,0.0565, 0)
#l=ROOT.TLine(c.GetUxmin(),0.0515,c.GetUxmax(),0.0515) #0.0515+(3*0.005)=
#pt = ROOT.TPave(0,0.0467,1.4,0.0563, 0)
#l=ROOT.TLine(c.GetUxmin(),0.0515,c.GetUxmax(),0.0515) #0.0515+(3*0.0048)=
#l=ROOT.TLine(c.GetUxmin(),0.0256,c.GetUxmax(),0.0256) #0.0256+(3*0.0017)=
#pt = ROOT.TPave(0,0.0239,1.4,0.0273, 0)
#l=ROOT.TLine(c.GetUxmin(),0.0246,c.GetUxmax(),0.0246)#0.0246+(3*0.0015)
#pt = ROOT.TPave(0,0.0231,1.4,0.0261, 0)
l=ROOT.TLine(c.GetUxmin(),0.0244,c.GetUxmax(),0.0244) #0.0244+(3*0.0015)=
pt = ROOT.TPave(0,0.0229,1.4,0.0259, 0)#
pt.Draw("SAME")
pt.SetFillStyle(3004)
pt.SetFillColor(9)#40#7#9
l.SetLineColor(ROOT.kBlack)
l.Draw("SAME")


c.Draw()
#c.BuildLegend()

**Xi(1320) with FDe - mc - BW - fit**

In [25]:
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p1p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p2p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p3p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p35p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p4p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p5p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p6p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p65p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p7p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p8p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p9p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p2p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p4p_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")

h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)
#fn = ROOT.TF1("fn", ROOT.Voigt, 0.5, 2.5, 3)#median, sigma, lg Voigt
fitFcn = ROOT.TF1("fitFcn", ROOT.fitBW, 0.5, 2.0, 5)
#fitFcn = ROOT.TF1("fitFcn", ROOT.fitGaus, 0.5, 2.0, 5)
#fitFcn = ROOT.TF1("fitFcn", ROOT.fitGaus, 0.5, 2.0, 6)
#fitFcn = ROOT.TF1("fitFcn", ROOT.p2fitDoubleGaus, 0.5, 2.0, 9)
fitFcn.SetLineColor(ROOT.kMagenta)#kMagenta#kBlue
fitFcn.SetParameter(1, 1.322)
fitFcn.SetParameter(2, 0.02)


%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
#hmc.Draw("SAME")

#fitResult = h.Fit("fitFcn", "S", " ", 1.1, 1.7)#1.2, 1.7
#fitResult = h.Fit("fitFcn", "S", " ", 1.1, 1.7)
fitResult = h.Fit("fitFcn", "S", " ", 1.0, 2.0)
covMatrix = fitResult.GetCovarianceMatrix()
sigXi1320_covMatrix = covMatrix.GetSub(0, 2, 0, 2, " ")
bg_covMatrix = covMatrix.GetSub(3, 4, 3, 4, " ")

#h.Fit("fn", "S", " ", 1.26, 1.4)
#h.Fit("gaus", "S", " ", 1.285, 1.385)
#fitFcn = h.GetFunction("gaus") # fit function

xsq = fitFcn.GetChisquare()
ndf = fitFcn.GetNDF()
chi2perndf = xsq/ndf

fitFcn = h.GetFunction("fitFcn")

factor = 3
lcut = fitFcn.GetParameter(1) - factor*fitFcn.GetParameter(2)
ucut = fitFcn.GetParameter(1) + factor*fitFcn.GetParameter(2)
print(f"Fit mean:: {fitFcn.GetParameter(1):.5f} Fit sigma:: {fitFcn.GetParameter(2):.5f} lcut:: {lcut:.5f} ucut:: {ucut:.5f}")
#pm6sigmav0 :: 1.13124<=mm_fkpskp && mm_fkpskp<=1.54534
#pm6sigma :: 1.18448<=mm_fkpskp && mm_fkpskp<=1.49119
#pm3sigmav0 :: 1.23477<=mm_fkpskp && mm_fkpskp<=1.44182
#pm3sigmav1 :: 1.26116<=mm_fkpskp && mm_fkpskp<=1.41451
#pm3sigmav2 :: 1.26470<=mm_fkpskp && mm_fkpskp<=1.41230


#improve the picture:
backFcn = ROOT.TF1("backFcn", ROOT.background, 1.0, 1.6, 2)
backFcn.SetParameters(fitFcn.GetParameter(3), fitFcn.GetParameter(4))
backFcn.SetLineColor(ROOT.kRed)
backFcn.SetLineStyle(2)
#signalFcnXi1320 = ROOT.TF1("signalFcnXi1320", ROOT.GaussianPeak, 1.0, 1.6, 3)
signalFcnXi1320 = ROOT.TF1("signalFcnXi1320", ROOT.lorentzianPeak, 1.0, 1.6, 3)
signalFcnXi1320.SetParameters(fitFcn.GetParameter(0), fitFcn.GetParameter(1), fitFcn.GetParameter(2))
signalFcnXi1320.SetLineColor(ROOT.kBlue)#ROOT.kGreen
backFcn.Draw("same")
signalFcnXi1320.Draw("same")

#Intergal from (3sigma)signal and background function
factor = 3
Intg1 = signalFcnXi1320.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2))
Intgb = backFcn.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2))
#Yield per bin width
binw = h.GetBinWidth(1)
Yield1 = Intg1/(binw)
bg = Intgb/(binw)

err_Intg1 = signalFcnXi1320.IntegralError(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2), signalFcnXi1320.GetParameters(), sigXi1320_covMatrix.GetMatrixArray())
err_xi1320_yield = err_Intg1/(binw)


startx=h.GetXaxis().GetXmin()+0.05*(h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())
starty10=1.0*h.GetMaximum()
starty=0.9*h.GetMaximum()
starty1=0.8*h.GetMaximum()
starty30=0.7*h.GetMaximum()
starty2=0.6*h.GetMaximum()
starty3=0.5*h.GetMaximum()
starty0=0.4*h.GetMaximum()

summ= ROOT.TLatex(startx, starty, f"#mu1:{fitFcn.GetParameter(1):5.4f}#pm{fitFcn.GetParError(1):5.4f}")
sum1= ROOT.TLatex(startx, starty1, f"#sigma1:{fitFcn.GetParameter(2):5.4f}#pm{fitFcn.GetParError(2):5.4f}")
sum10= ROOT.TLatex(startx, starty10, f"Yield1:{Yield1:.0f}#pm{err_xi1320_yield:.0f}")
sum4= ROOT.TLatex(startx, starty0, f"#chi2/Ndf:{chi2perndf:5.2f}")
#"""

summ.SetTextSize(0.05)#0.065
summ.SetTextColor(ROOT.kBlue)
summ.Draw("same")
#"""
sum1.SetTextSize(0.05)
sum1.SetTextColor(ROOT.kBlue)
sum1.Draw("same")
sum10.SetTextSize(0.05)
sum10.SetTextColor(ROOT.kBlue)
sum10.Draw("same")
sum4.SetTextSize(0.05)
sum4.SetTextColor(ROOT.kMagenta)
sum4.Draw("same")

l=ROOT.TLine(1.322, 0, 1.322, 500)
l.SetLineColor(1)
l.SetLineStyle(10)
#l.Draw("same")
l2=ROOT.TLine(1.532, 0, 1.532, 500)
l2.SetLineColor(1)
l2.SetLineStyle(10)
#l2.Draw("same")

legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend1.SetHeader("fall2018-out-fde","C") # option "C" allows to center the header
#legend1.AddEntry("mc","mc","f")
#legend1.AddEntry("data","data","lpe")#lpe
#legend1.AddEntry("mc","mc-psmear-non","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.1%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.2%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.3%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.35%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.4%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.5%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.6%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.65%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.7%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.8%","lpe")
#legend1.AddEntry("mc","mc-psmeard-0.9%","lpe")
#legend1.AddEntry("mc","mc-psmeard-1%","lpe")
#legend1.AddEntry("mc","mc-psmeard-1.2%","lpe")
legend1.AddEntry("mc","mc-psmeard-1.4%","lpe")

legend1.SetTextFont(72)
legend1.SetTextSize(0.8)
legend1.Draw()

c.Draw()

Fit mean:: 1.33427 Fit sigma:: 0.06998 lcut:: 1.12433 ucut:: 1.54422
 FCN=1950.44 FROM MIGRAD    STATUS=CONVERGED     406 CALLS         407 TOTAL
                     EDM=1.10757e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.4 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           8.10455e+02   4.60281e+00  -1.83621e-03  -8.73398e-06
   2  p1           1.33427e+00   2.83544e-04   6.58297e-08   1.71928e-02
   3  p2           6.99812e-02   5.19176e-04   2.55042e-07   7.89723e-02
   4  p3          -2.50977e+02   5.62984e+00  -3.01857e-03  -3.97033e-06
   5  p4           1.40901e+02   3.34397e+00   1.77245e-03   4.90757e-06


In [26]:
p_smear = array('d', [0, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.65, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4])
err_p_smear = array('d', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#chi2 = [5.40, 5.42, 5.11, 5.07, 4.84, 4.63, 4.23, 3.49, 3.42, 2.65, 1.84, 1.69]
sig_xi1320 = array('d', [0.0337,0.0343, 0.0373, 0.0416, 0.0452, 0.0473, 0.0517, 0.0577, 0.0614, 0.0639, 0.0700, 0.0761, 0.0837, 0.1002, 0.1163])
err_sig_xi1320 = array('d', [0.0002,0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0004, 0.0004, 0.0005, 0.0005, 0.0005, 0.0006, 0.0006, 0.0008, 0.0009])
gr = ROOT.TGraphErrors(len(p_smear), p_smear, sig_xi1320, err_p_smear, err_sig_xi1320)
#gr = ROOT.TGraph(len(p_smear), p_smear, sig_lam)
gr.SetTitle("#Xi^{-}(1320) width in MM(eK^{+}K^{+}) from MC (smeared))")
#gr.SetTitle("#sigma(of #Xi^{-}(1320)fit from MC)")
gr.GetXaxis().SetTitle("Gaussian smearing factor #Delta p/p(%)")
gr.GetYaxis().SetTitle("#sigma [GeV/c^{2}]")
gr.SetMarkerStyle(21)
gr.SetMarkerSize(0.8)
gr.SetLineColor(ROOT.kRed)
gr.SetMarkerColor(ROOT.kGreen)
gr.SetFillStyle(1001)

%jsroot on
ROOT.gStyle.SetOptTitle(1)
c = ROOT.TCanvas()
gr.Draw("AC*")

#hbox = ROOT.TH2F("hbox(name)","Option BOX example(title)",3(nbinsx),0(xlow),3(xup),3(nbinsy),0(ylow),3(yup))# did not work
#band = ROOT.TH2F("band", "",1 , c.GetUxmin(), gr.GetXaxis().GetXmax(), 1, 0.465, 0.0565)
#band.SetFillColor(42)
#band.Fill(1, 0.0515, 0.01)
#band.Draw("SAME")
"""
multigraph = ROOT.TMultiGraph()
# lower bound
x1 = array('d', [0,1])
y1 = array('d', [0.465,0.465])
excl1 = ROOT.TGraph(5,x1,y1)
excl1.SetLineColor(41)
excl1.SetFillColor(41)
excl1.SetFillStyle(1001)
# upper bound
x1 = array('d', [0,1])
y2 = array('d', [0.0565,0.0565])
excl2 = ROOT.TGraph(5,x1,y2)
excl2.SetLineColor(46)
excl2.SetFillColor(46)
#excl2.SetFillStyle(1001)

multigraph.Add(excl1,"LF")
multigraph.Add(excl2,"LF")
#multigraph.Draw("same")
"""

t1 = ROOT.TLatex(0.8, 0.057, "#Xi^{-}(1320) width from data")
t1.Draw("SAME")

#pt = ROOT.TPaveText(0,0.0465,1.4,0.0565)
#pt.AddText("")
#pt.SetTextColor(1)
#pt.SetTextSize(0.04)
#pt.SetShadowColor(ROOT.kGray)
#pt = ROOT.TPave(0,0.0465,1.4,0.0565, 0)
#l=ROOT.TLine(c.GetUxmin(),0.0515,c.GetUxmax(),0.0515) #0.0515+(3*0.005)=
#pt = ROOT.TPave(0,0.0467,1.4,0.0563, 0)
#l=ROOT.TLine(c.GetUxmin(),0.0515,c.GetUxmax(),0.0515) #0.0515+(3*0.0048)=
pt = ROOT.TPave(0,0.0422,1.4,0.05, 0)
l=ROOT.TLine(c.GetUxmin(),0.0461,c.GetUxmax(),0.0461) #0.0461+(3*0.0039)=
pt.Draw("SAME")
pt.SetFillStyle(3004)
pt.SetFillColor(9)#40#7#9
l.SetLineColor(ROOT.kBlack)
l.Draw("SAME")


c.Draw()
#c.BuildLegend()

**Xi(1320) with FDe - data**

In [38]:
h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fde.root").Histo1D(("data", "data", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#100, 1.2, 1.7#250, 0.5, 3.0#400, 0.5, 2.5#250, 0.5, 2.5#150, 0.5, 2.6#
#MC files
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)
#fn = ROOT.TF1("fn", ROOT.Voigt, 1.0, 2.5, 3)#median, sigma, lg
#fitFcn = ROOT.TF1("fitFcn", ROOT.fitBW, 0.5, 2.0, 5)
#fitFcn = ROOT.TF1("fitFcn", ROOT.fitGaus, 0.5, 2.0, 6)
fitFcn = ROOT.TF1("fitFcn", ROOT.p2fitDoubleGaus, 0.5, 2.0, 9)#(fit range)1.08, 1.9#1.2, 1.9#1.54(0.08) fit converged
#fitFcn = ROOT.TF1("fitFcn", ROOT.p2fitDoubleBW, 0.5, 2.0, 9)#1.2, 1.9#1.53(0.04)// 1.12, 1.9#1.53(0.04) fit converged
fitFcn.SetLineColor(ROOT.kMagenta)#kMagenta#kBlue
#fitFcn.SetParameter(0, 1.322)
fitFcn.SetParameter(1, 1.322)
fitFcn.SetParameter(2, 0.03)
fitFcn.SetParameter(4, 1.54)
fitFcn.SetParameter(5, 0.04)

%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
#hmc.Draw("SAME")

fitResult = h.Fit("fitFcn", "EMS", " ", 1.06, 1.9)#1.06 to 1.9 1.4
#fitResult = h.Fit("fitFcn", "S", " ", 1.2, 1.9)
covMatrix = fitResult.GetCovarianceMatrix()
sigXi1320_covMatrix = covMatrix.GetSub(0, 2, 0, 2, " ")
sigXi1530_covMatrix = covMatrix.GetSub(3, 5, 3, 5, " ")
bg_covMatrix = covMatrix.GetSub(6, 8, 6, 8, " ")

#h.Fit("fn", "S", " ", 1.26, 1.4)
#h.Fit("gaus", "S", " ", 1.285, 1.385)
#fitFcn = h.GetFunction("gaus") # fit function

xsq = fitFcn.GetChisquare()
ndf = fitFcn.GetNDF()
chi2perndf = xsq/ndf

fitFcn = h.GetFunction("fitFcn")

factor = 3
lcut = fitFcn.GetParameter(1) - factor*fitFcn.GetParameter(2)
ucut = fitFcn.GetParameter(1) + factor*fitFcn.GetParameter(2)
print(f"Fit mean:: {fitFcn.GetParameter(1):.5f} Fit sigma:: {fitFcn.GetParameter(2):.5f} lcut:: {lcut:.5f} ucut:: {ucut:.5f}")
#pm6sigmav0 :: 1.13124<=mm_fkpskp && mm_fkpskp<=1.54534
#pm6sigma :: 1.18448<=mm_fkpskp && mm_fkpskp<=1.49119
#pm3sigmav0 :: 1.23477<=mm_fkpskp && mm_fkpskp<=1.44182
#pm3sigmav1 :: 1.26116<=mm_fkpskp && mm_fkpskp<=1.41451
#pm3sigmav2 :: 1.26470<=mm_fkpskp && mm_fkpskp<=1.41230

h_signal_xi1320 = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fde.root").Filter("1.26116<=mm_fkpskp && mm_fkpskp<=1.41451").Histo1D(("dataxi1320", "dataxi1320", 100, 0.5, 2.5), "mm_fkpskp")
h_signal_xi1320.SetTitle("MM(eK^{+}K^{+})")
#h_signal_xi1320.SetTitleSize(0.06, "x")
h_signal_xi1320.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h_signal_xi1320.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h_signal_xi1320.GetXaxis().SetTitleSize(0.05)
h_signal_xi1320.GetYaxis().SetTitleSize(0.05)
h_signal_xi1320.GetXaxis().SetTitleOffset(0.9)
#h_signal_xi1320.GetYaxis().SetTitleOffset(0.9)


#improve the picture:
backFcn = ROOT.TF1("backFcn", ROOT.p2background, 1.0, 1.95, 3)
backFcn.SetParameters(fitFcn.GetParameter(6), fitFcn.GetParameter(7), fitFcn.GetParameter(8))
backFcn.SetLineColor(ROOT.kRed)
backFcn.SetLineStyle(2)
signalFcnXi1320 = ROOT.TF1("signalFcnXi1320", ROOT.GaussianPeak, 1.0, 1.9, 3)
#signalFcnXi1320 = ROOT.TF1("signalFcnXi1320", ROOT.lorentzianPeak, 1.0, 1.9, 3)
signalFcnXi1320.SetParameters(fitFcn.GetParameter(0), fitFcn.GetParameter(1), fitFcn.GetParameter(2))
signalFcnXi1320.SetLineColor(ROOT.kBlue)#ROOT.kGreen
signalFcnXi1530 = ROOT.TF1("signalFcnXi1530", ROOT.Gaussian, 1.2, 1.9, 3)
#signalFcnXi1530 = ROOT.TF1("signalFcnXi1530", ROOT.lorentzian, 1.2, 1.9, 3)
signalFcnXi1530.SetParameters(fitFcn.GetParameter(3), fitFcn.GetParameter(4), fitFcn.GetParameter(5))
signalFcnXi1530.SetLineColor(ROOT.kGreen)#ROOT.kBlue
#signalFcn.SetNpx(500)
#h1_fitFcn.Draw("same")
backFcn.Draw("same")
signalFcnXi1320.Draw("same")
#signalFcnXi1530.Draw("same")

fitFcn.Draw("same")

#Intergal from (3sigma)signal and background function
factor = 3
Intg1 = signalFcnXi1320.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2))
Intg2 = signalFcnXi1530.Integral(fitFcn.GetParameter(4)-factor*fitFcn.GetParameter(5), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5))
Intgb = backFcn.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5))
#Yield per bin width
binw = h.GetBinWidth(1)
Yield1 = Intg1/(binw)
Yield2 = Intg2/(binw)
bg = Intgb/(binw)

err_Intg1 = signalFcnXi1320.IntegralError(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2), signalFcnXi1320.GetParameters(), sigXi1320_covMatrix.GetMatrixArray())
err_xi1320_yield = err_Intg1/(binw)

err_Intg2 = signalFcnXi1530.IntegralError(fitFcn.GetParameter(4)-factor*fitFcn.GetParameter(5), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5), signalFcnXi1530.GetParameters(), sigXi1530_covMatrix.GetMatrixArray())
err_xi1530_yield = err_Intg2/(binw)

startx=h.GetXaxis().GetXmin()+0.05*(h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())
starty10=1.0*h.GetMaximum()
starty=0.9*h.GetMaximum()
starty1=0.8*h.GetMaximum()
starty30=0.7*h.GetMaximum()
starty2=0.6*h.GetMaximum()
starty3=0.5*h.GetMaximum()
starty0=0.4*h.GetMaximum()

summ= ROOT.TLatex(startx, starty, f"#mu:{fitFcn.GetParameter(1):5.4f}#pm{fitFcn.GetParError(1):5.4f}")
sum1= ROOT.TLatex(startx, starty1, f"#sigma:{fitFcn.GetParameter(2):5.4f}#pm{fitFcn.GetParError(2):5.4f}")
sum10= ROOT.TLatex(startx, starty10, f"Yield:{Yield1:.0f}#pm{err_xi1320_yield:.0f}")
sum2= ROOT.TLatex(startx, starty2, f"mu2:{fitFcn.GetParameter(4):5.4f}#pm{fitFcn.GetParError(4):5.4f}")
sum3= ROOT.TLatex(startx, starty3, f"#sigma2:{fitFcn.GetParameter(5):5.4f}#pm{fitFcn.GetParError(5):5.4f}")
sum30= ROOT.TLatex(startx, starty30, f"Yield2:{Yield2:.0f}#pm{err_xi1530_yield:.0f}")
sum4= ROOT.TLatex(startx, starty0, f"#chi2/Ndf:{chi2perndf:5.2f}")
#"""

summ.SetTextSize(0.05)#0.065
summ.SetTextColor(ROOT.kBlue)
summ.Draw("same")
#"""
sum1.SetTextSize(0.05)
sum1.SetTextColor(ROOT.kBlue)
sum1.Draw("same")
sum10.SetTextSize(0.05)
sum10.SetTextColor(ROOT.kBlue)
sum10.Draw("same")
sum2.SetTextSize(0.05)
sum2.SetTextColor(ROOT.kGreen)
#sum2.Draw("same")
sum3.SetTextSize(0.05)
sum3.SetTextColor(ROOT.kGreen)
#sum3.Draw("same")
sum30.SetTextSize(0.05)
sum30.SetTextColor(ROOT.kGreen)
#sum30.Draw("same")
sum4.SetTextSize(0.05)
sum4.SetTextColor(ROOT.kMagenta)
sum4.Draw("same")

l=ROOT.TLine(1.322, 0, 1.322, 540)
l.SetLineColor(1)
l.SetLineStyle(10)
#l.Draw("same")
l2=ROOT.TLine(1.532, 0, 1.532, 540)
l2.SetLineColor(1)
l2.SetLineStyle(10)
#l2.Draw("same")

legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
#legend1.SetHeader("fall2018-out-fde","C") # option "C" allows to center the header
#legend1.AddEntry("mc","mc","f")
legend1.AddEntry("data","Data-fall2018-out-fde","pe")#lpe
legend1.AddEntry("fitFcn","Sig. + Bg. Fit","l")
legend1.AddEntry("backFcn", "Bg. Fit","l")
legend1.AddEntry("signalFcnXi1320", "#Xi(1320)","l")
legend1.SetTextFont(72)
legend1.SetTextSize(0.8)
legend1.Draw()

c.Draw()

Fit mean:: 1.33830 Fit sigma:: 0.02440 lcut:: 1.26511 ucut:: 1.41149
 FCN=70.7802 FROM MINOS     STATUS=SUCCESSFUL     70 CALLS        3167 TOTAL
                     EDM=1.23894e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.03486e+02   1.77219e+01  -4.48862e-02  -1.61047e-05
   2  p1           1.33830e+00   1.46093e-03   3.82643e-06  -5.79133e-02
   3  p2           2.43961e-02   1.51271e-03   1.18477e-05  -9.82319e-01
   4  p3           7.88730e+01   1.67753e+01  -9.33759e-02   1.77457e-05
   5  p4           1.54751e+00   5.39133e-03   3.94542e-05   1.66797e-01
   6  p5           1.90633e-02   4.59015e-03   6.69338e-05   2.32706e-01
   7  p6          -4.86717e+02   9.65357e+01   2.66504e-02   3.96132e-04
   8  p7           6.31804e+02   1.38263e+02  -3.58517e-02   5.71278e-04
   9  p8          -1.04493e+02   4.77824e+01  

**Background subtraction from parametrized bg polynimial function**

In [28]:
%jsroot on
ROOT.gStyle.SetOptStat(00000)
ROOT.gStyle.SetOptTitle(0)
c = ROOT.TCanvas()
#h_bgCorrect = h_signal_xi1320.Clone()
#h_bgCorrect = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fde.root").Histo1D(("data", "data", 100, 0.5, 2.5), "mm_fkpskp")
h_bgCorrect = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("data", "data", 100, 0.5, 2.5), "mm_fkpskp")
h_bgCorrect.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h_bgCorrect.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h_bgCorrect.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h_bgCorrect.GetXaxis().SetTitleSize(0.05)
h_bgCorrect.GetYaxis().SetTitleSize(0.05)
h_bgCorrect.GetXaxis().SetTitleOffset(0.9)

bgval = backFcn.Eval(h_bgCorrect.GetXaxis().GetBinCenter(1))
print("bg value in bin no. seven from parametrized fit fcn= ", bgval)
print("Total no of bins::", h_bgCorrect.GetNbinsX())
for i in range(h_bgCorrect.GetNbinsX()):
    binCenter = h_bgCorrect.GetXaxis().GetBinCenter(i+1)
    bgval = backFcn.Eval(binCenter)
    sigbg = h_bgCorrect.GetBinContent(i+1)
    if bgval >=70 and bgval<sigbg:#bgval >=110#bgval >=70
        bgvalErr = math.sqrt(bgval)
        sigbgErr = h_bgCorrect.GetBinError(i+1) #math.sqrt(sigbg)
        #corErr = math.sqrt(sigbgErr*sigbgErr + bgvalErr*bgvalErr)
        corErr = math.sqrt(sigbg + bgval)
        h_bgCorrect.SetBinContent(i+1, sigbg-bgval)
        h_bgCorrect.SetBinError(i+1, corErr)
        
    elif bgval>sigbg:
        h_bgCorrect.SetBinContent(i+1, 0)
    else:
        h_bgCorrect.SetBinContent(i+1, 0)

c.cd(3)
h_bgCorrect.Draw("E")
h_bgCorrect.SetLineColor(ROOT.kRed)
h_bgCorrect.SetMarkerStyle(21)
h_bgCorrect.SetMarkerSize(0.8)
c.Draw()

bg value in bin no. seven from parametrized fit fcn=  -191.6758697280089
Total no of bins:: 100


**Xi(1320) with FDe MC Smearing VS Data**

In [29]:
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fde.root").Filter("1.26116<=mm_fkpskp && mm_fkpskp<=1.41451").Histo1D(("data", "data", 100, 0.5, 2.5), "mm_fkpskp")
h = h_bgCorrect.Clone()
#100, 1.2, 1.7#250, 0.5, 3.0(10)#400, 0.5, 2.5(5)#250, 0.5, 2.5(8)#150, 0.5, 2.6(14)#100, 0.5, 2.5(20)
h.Scale(1/h.Integral())
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
#h.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h.GetYaxis().SetTitle("Counts")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

#hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")#jose simulated files
#hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fall2018_out_fde_test.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.46").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p1p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p2p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p3p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p35p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p4p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p5p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p6p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p65p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p7p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p8p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared0p9p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p2p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_smeared1p4p_fall2018out_fde.root").Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.48").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")

#.Filter("1.2<=mm_fkpskp && mm_fkpskp<=1.46")#Filter("1.26116<=mm_fkpskp && mm_fkpskp<=1.41451")

hmc.Scale(1/hmc.Integral())
hmc.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
hmc.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
#hmc.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
hmc.GetYaxis().SetTitle("Counts")
hmc.GetXaxis().SetTitleSize(0.05)
hmc.GetYaxis().SetTitleSize(0.05)
hmc.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

print(f"Integral of 3Kaon MM from data and mc::, {h.Integral()} and {hmc.Integral()}")
gof_test = 0.0
for i in range(h.GetNbinsX()):
    #print(f"bin no. {i+1} has normalized data, {h2.GetBinContent(i+1)} and normalized MC, {h1.GetBinContent(i+1)}")
    data = h.GetBinContent(i+1)
    err = h.GetBinError(i+1)
    mc = hmc.GetBinContent(i+1)
    if data>0 and mc>0:
        gof_test += math.pow(data-mc, 2)/math.pow(err, 2)
        #gof_test = math.pow(mc-data, 2)*10000
            
#print(f"Sum of squares (*10000):: {gof_test:.2f}")
print(f"Chi squares:: {gof_test:.2f}")

%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
#hmc.Draw("HIST")
#h.Draw("SAMEE")
h.Draw("E")
hmc.Draw("SAMEHIST")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
#h1.SetStats(0)
#h.Fit("gaus", "S", " ", 1.285, 1.385)
#h.Fit("fn", "S", " ", 1.26, 1.4)

t1 = ROOT.TLatex(.6, 0.1, f"Chi square = {gof_test:0.2f}")
t1.Draw()

legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend1.SetHeader("#Xi^{-}(1320) fall2018-out-fde","C") # option "C" allows to center the header
legend1.AddEntry("data","data","lpe")#lpe
#legend1.AddEntry("mc","mc-psmear-non","f")
#legend1.AddEntry("mc","mc-psmeard-0.1%","f")
#legend1.AddEntry("mc","mc-psmeard-0.2%","f")
legend1.AddEntry("mc","mc-psmeard-0.25%","f")###
#legend1.AddEntry("mc","mc-psmeard-0.3%","f")
#legend1.AddEntry("mc","mc-psmeard-0.35%","f")
#legend1.AddEntry("mc","mc-psmeard-0.4%","f")
#legend1.AddEntry("mc","mc-psmeard-0.5%","f")
#legend1.AddEntry("mc","mc-psmeard-0.6%","f")
#legend1.AddEntry("mc","mc-psmeard-0.65%","f")
#legend1.AddEntry("mc","mc-psmeard-0.7%","f")
#legend1.AddEntry("mc","mc-psmeard-0.8%","f")
#legend1.AddEntry("mc","mc-psmeard-0.9%","f")
#legend1.AddEntry("mc","mc-psmeard-1%","f")
#legend1.AddEntry("mc","mc-psmeard-1.2%","f")
#legend1.AddEntry("mc","mc-psmeard-1.4%","f")
legend1.SetTextFont(72)
legend1.SetTextSize(0.8)
legend1.Draw()
c.Draw()

Integral of 3Kaon MM from data and mc::, 1.0 and 0.9999999999999998
Chi squares:: 27.69


**Chi2 Minimization DATA VS MC(smeared) Comparision Xi(1320)**

In [30]:
p_smear = array('d', [0, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.65, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4])
#err_p_smear = array('d', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#chi2 = array('d', [234.01, 217.84, 182.92, 150.48, 140.12, 125.43, 113.78, 108.03, 114.14, 106.70, 110.36, 118.40, 121.10, 144.49, 167.55])
chi2 = array('d', [74.81, 68.55, 52.74, 40.77, 38.12, 33.41, 34.04, 38.75, 47.14, 45.93, 55.49, 66.32, 74.68, 100.65, 123.48])
#err_sig_xi1320 = array('d', [0.0002,0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0004, 0.0004, 0.0005, 0.0005, 0.0005, 0.0006, 0.0006, 0.0008, 0.0009])
#gr = ROOT.TGraphErrors(len(p_smear), p_smear, sig_xi1320, err_p_smear, err_sig_xi1320)
gr = ROOT.TGraph(len(p_smear), p_smear, chi2)
gr.SetTitle("MM(eK^{+}K^{+}) Data(Fall2018outfde)/MC(smeared) matching")
gr.GetXaxis().SetTitle("Gaussian smearing factor #Delta p/p(%)")
gr.GetYaxis().SetTitle("Chi square")
gr.SetMarkerStyle(21)
gr.SetMarkerSize(0.8)
gr.SetLineColor(ROOT.kRed)
gr.SetMarkerColor(ROOT.kGreen)

%jsroot on
ROOT.gStyle.SetOptTitle(1)
c = ROOT.TCanvas()
gr.Draw("AC*")

#l=ROOT.TLine(c.GetUxmin(),0.0256,c.GetUxmax(),0.0256) #0.0256+(3*0.0017)=
l=ROOT.TLine(c.GetUxmin(),0.0461,c.GetUxmax(),0.0461) #0.0515+(3*0.005)=
#0.0461+(3*0.0039)
l.SetLineColor(ROOT.kRed)
#l.Draw("SAME")

c.Draw()
#c.BuildLegend()

**Xi(1320) with FTe - mc - gaussian fit**

In [31]:
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p1p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p2p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p3p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p35p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p4p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p5p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p6p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p65p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p7p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p8p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared0p9p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared1p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared1p2p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared1p4p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared1p6p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared1p8p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared2p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared2p2p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared2p4p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared2p6p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared2p8p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared3p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared3p2p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared3p4p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared3p6p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared3p8p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared4p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fte/plot/rec_xi_1320_mc_smeared4p2p_fall2018out_fte.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")

h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(1)
c = ROOT.TCanvas()
h.Draw("E")
#h.Fit("gaus", "S", " ", 1.24, 1.4)
#h.Fit("gaus", "S", " ", 1.26, 1.4)
#h.Fit("gaus", "S", " ", 1.24, 1.42)#0p9p
#h.Fit("gaus", "S", " ", 1.2, 1.48)#1p2p
h.Fit("gaus", "S", " ", 1.15, 1.48)#1p2p
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
c.Draw()

 FCN=1371 FROM MIGRAD    STATUS=CONVERGED      58 CALLS          59 TOTAL
                     EDM=5.07509e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     9.91827e+03   5.30284e+01   6.97523e-01  -6.37609e-07
   2  Mean         1.33075e+00   2.10798e-04   3.79785e-06  -1.01222e+00
   3  Sigma        5.33460e-02   2.00659e-04   1.62297e-05  -1.94225e-01


In [32]:
#p_smear = array('d', [0, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.65, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2,3.4,3.6,3.8,4.0,4.2])
#err_p_smear = array('d', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0])
#sig_xi1320 = array('d', [0.03302,0.03310, 0.03308, 0.03329, 0.03327, 0.03351, 0.033370, 0.03388, 0.03397, 0.03408, 0.03460, 0.03619, 0.03914, 0.04017, 0.04133, 0.04255, 0.04405, 0.04537, 0.04708,0.04809, 0.04946, 0.05098, 0.05218, 0.05394, 0.05465, 0.05593, 0.05949, 0.06083, 0.06252])
#err_sig_xi1320 = array('d', [0.00013,0.00013, 0.00013, 0.00013, 0.00013, 0.00014, 0.00014, 0.00014, 0.00014, 0.00014, 0.00014, 0.00014, 0.00016, 0.00016, 0.00016, 0.00017, 0.00017, 0.00018, 0.00018, 0.00018, 0.00019, 0.00020, 0.00020, 0.00021, 0.00021, 0.00022, 0.00023, 0.00023, 0.00024])

p_smear = array('d', [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2,3.4,3.6,3.8,4.0,4.2])
err_p_smear = array('d', [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0])
sig_xi1320 = array('d', [0.03627, 0.03627, 0.03688, 0.03732, 0.03808, 0.03914, 0.04017, 0.04133, 0.04255, 0.04405, 0.04537, 0.04708,0.04809, 0.04946, 0.05098, 0.05335, 0.05525, 0.05617, 0.05757, 0.05949, 0.06083, 0.06252])
err_sig_xi1320 = array('d', [0.00015, 0.00015, 0.00015, 0.00015, 0.00015, 0.00016, 0.00016, 0.00016, 0.00017, 0.00017, 0.00018, 0.00018, 0.00018, 0.00019, 0.00020, 0.00020, 0.00021, 0.00021, 0.00022, 0.00023, 0.00023, 0.00024])
gr = ROOT.TGraphErrors(len(p_smear), p_smear, sig_xi1320, err_p_smear, err_sig_xi1320)
#gr = ROOT.TGraph(len(p_smear), p_smear, sig_lam)
gr.SetTitle("#Xi^{-}(1320) width in MM(eK^{+}K^{+}) from MC (smeared))")
#gr.SetTitle("#sigma(of #Xi^{-}(1320)fit from MC)")
gr.GetXaxis().SetTitle("Gaussian smearing factor for FT-e #Delta p/p(%)")
gr.GetYaxis().SetTitle("#sigma [GeV/c^{2}]")
gr.SetMarkerStyle(21)
gr.SetMarkerSize(0.8)
gr.SetLineColor(ROOT.kRed)
gr.SetMarkerColor(ROOT.kGreen)
gr.SetFillStyle(1001)

%jsroot on
ROOT.gStyle.SetOptTitle(1)
c = ROOT.TCanvas()
gr.Draw("AC*")

#hbox = ROOT.TH2F("hbox(name)","Option BOX example(title)",3(nbinsx),0(xlow),3(xup),3(nbinsy),0(ylow),3(yup))# did not work
#band = ROOT.TH2F("band", "",1 , c.GetUxmin(), gr.GetXaxis().GetXmax(), 1, 0.465, 0.0565)
#band.SetFillColor(42)
#band.Fill(1, 0.0515, 0.01)
#band.Draw("SAME")
"""
multigraph = ROOT.TMultiGraph()
# lower bound
x1 = array('d', [0,1])
y1 = array('d', [0.465,0.465])
excl1 = ROOT.TGraph(5,x1,y1)
excl1.SetLineColor(41)
excl1.SetFillColor(41)
excl1.SetFillStyle(1001)
# upper bound
x1 = array('d', [0,1])
y2 = array('d', [0.0565,0.0565])
excl2 = ROOT.TGraph(5,x1,y2)
excl2.SetLineColor(46)
excl2.SetFillColor(46)
#excl2.SetFillStyle(1001)

multigraph.Add(excl1,"LF")
multigraph.Add(excl2,"LF")
#multigraph.Draw("same")
"""

t1 = ROOT.TLatex(0.8, 0.05, "#Xi^{-}(1320) width from data")
t1.Draw("SAME")

pt = ROOT.TPave(0,0.0411,4.5,0.0553, 0)
l=ROOT.TLine(c.GetUxmin(),0.0482,4.5,0.0482)#0.0482+(3*0.0071)
pt.Draw("SAME")
pt.SetFillStyle(3004)
pt.SetFillColor(9)#40#7#9
l.SetLineColor(ROOT.kBlack)
l.Draw("SAME")


c.Draw()
#c.BuildLegend()

**Xi(1320) with FTe - data**

In [33]:
h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 100, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","../simu_prod/xi_1320_mc_fall2018out_fde/plot/rec_xi_1320_mc_fall2018out_fde.root").Histo1D(("mc", "mc", 100, 0.5, 2.5), "mm_fkpskp")
#100, 1.2, 1.7#250, 0.5, 3.0#400, 0.5, 2.5#250, 0.5, 2.5#150, 0.5, 2.6#
#MC files
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)
#fn = ROOT.TF1("fn", ROOT.Voigt, 1.0, 2.5, 3)#median, sigma, lg
#fitFcn = ROOT.TF1("fitFcn", ROOT.fitBW, 0.5, 2.0, 5)
#fitFcn = ROOT.TF1("fitFcn", ROOT.fitGaus, 0.5, 2.0, 6)
fitFcn = ROOT.TF1("fitFcn", ROOT.p2fitDoubleGaus, 0.5, 2.0, 9)# fit converged
#fitFcn = ROOT.TF1("fitFcn", ROOT.p2fitDoubleBW, 0.5, 2.0, 9)# fit converged
fitFcn.SetLineColor(ROOT.kMagenta)#kMagenta#kBlue
#fitFcn.SetParameter(0, 1.322)
fitFcn.SetParameter(1, 1.33)
fitFcn.SetParameter(2, 0.02)
fitFcn.SetParameter(4, 1.56)
fitFcn.SetParameter(5, 0.4)

%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
#hmc.Draw("SAME")

fitResult = h.Fit("fitFcn", "EMS", " ", 1.1, 1.7)
#fitResult = h.Fit("fitFcn", "S", " ", 1.2, 1.9)
covMatrix = fitResult.GetCovarianceMatrix()
sigXi1320_covMatrix = covMatrix.GetSub(0, 2, 0, 2, " ")
sigXi1530_covMatrix = covMatrix.GetSub(3, 5, 3, 5, " ")
bg_covMatrix = covMatrix.GetSub(6, 8, 6, 8, " ")

#h.Fit("fn", "S", " ", 1.26, 1.4)
#h.Fit("gaus", "S", " ", 1.285, 1.385)
#fitFcn = h.GetFunction("gaus") # fit function

xsq = fitFcn.GetChisquare()
ndf = fitFcn.GetNDF()
chi2perndf = xsq/ndf

fitFcn = h.GetFunction("fitFcn")

factor = 3
lcut = fitFcn.GetParameter(1) - factor*fitFcn.GetParameter(2)
ucut = fitFcn.GetParameter(1) + factor*fitFcn.GetParameter(2)
print(f"Fit mean:: {fitFcn.GetParameter(1):.5f} Fit sigma:: {fitFcn.GetParameter(2):.5f} lcut:: {lcut:.5f} ucut:: {ucut:.5f}")
#pm6sigmav0 :: 1.13124<=mm_fkpskp && mm_fkpskp<=1.54534
#pm6sigma :: 1.18448<=mm_fkpskp && mm_fkpskp<=1.49119
#pm3sigmav0 :: 1.23477<=mm_fkpskp && mm_fkpskp<=1.44182
#pm3sigmav1 :: 1.26116<=mm_fkpskp && mm_fkpskp<=1.41451
#pm3sigmav2 :: 1.26470<=mm_fkpskp && mm_fkpskp<=1.41230

h_signal_xi1320 = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fde.root").Filter("1.26116<=mm_fkpskp && mm_fkpskp<=1.41451").Histo1D(("dataxi1320", "dataxi1320", 100, 0.5, 2.5), "mm_fkpskp")
h_signal_xi1320.SetTitle("MM(eK^{+}K^{+})")
#h_signal_xi1320.SetTitleSize(0.06, "x")
h_signal_xi1320.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h_signal_xi1320.GetYaxis().SetTitle("Counts/20 MeV/c^{2}")
h_signal_xi1320.GetXaxis().SetTitleSize(0.05)
h_signal_xi1320.GetYaxis().SetTitleSize(0.05)
h_signal_xi1320.GetXaxis().SetTitleOffset(0.9)
#h_signal_xi1320.GetYaxis().SetTitleOffset(0.9)


#improve the picture:
backFcn = ROOT.TF1("backFcn", ROOT.p2background, 1.1, 1.9, 3)
backFcn.SetParameters(fitFcn.GetParameter(6), fitFcn.GetParameter(7), fitFcn.GetParameter(8))
backFcn.SetLineColor(ROOT.kRed)
backFcn.SetLineStyle(2)
signalFcnXi1320 = ROOT.TF1("signalFcnXi1320", ROOT.GaussianPeak, 1.0, 1.9, 3)
#signalFcnXi1320 = ROOT.TF1("signalFcnXi1320", ROOT.lorentzianPeak, 1.0, 1.9, 3)
signalFcnXi1320.SetParameters(fitFcn.GetParameter(0), fitFcn.GetParameter(1), fitFcn.GetParameter(2))
signalFcnXi1320.SetLineColor(ROOT.kBlue)#ROOT.kGreen
signalFcnXi1530 = ROOT.TF1("signalFcnXi1530", ROOT.Gaussian, 1.2, 1.9, 3)
#signalFcnXi1530 = ROOT.TF1("signalFcnXi1530", ROOT.lorentzian, 1.2, 1.9, 3)
signalFcnXi1530.SetParameters(fitFcn.GetParameter(3), fitFcn.GetParameter(4), fitFcn.GetParameter(5))
signalFcnXi1530.SetLineColor(ROOT.kGreen)#ROOT.kBlue
#signalFcn.SetNpx(500)
#h1_fitFcn.Draw("same")
backFcn.Draw("same")
signalFcnXi1320.Draw("same")
signalFcnXi1530.Draw("same")

#Intergal from (3sigma)signal and background function
factor = 3
Intg1 = signalFcnXi1320.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2))
Intg2 = signalFcnXi1530.Integral(fitFcn.GetParameter(4)-factor*fitFcn.GetParameter(5), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5))
Intgb = backFcn.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5))
#Yield per bin width
binw = h.GetBinWidth(1)
Yield1 = Intg1/(binw)
Yield2 = Intg2/(binw)
bg = Intgb/(binw)

err_Intg1 = signalFcnXi1320.IntegralError(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2), signalFcnXi1320.GetParameters(), sigXi1320_covMatrix.GetMatrixArray())
err_xi1320_yield = err_Intg1/(binw)

err_Intg2 = signalFcnXi1530.IntegralError(fitFcn.GetParameter(4)-factor*fitFcn.GetParameter(5), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5), signalFcnXi1530.GetParameters(), sigXi1530_covMatrix.GetMatrixArray())
err_xi1530_yield = err_Intg2/(binw)

startx=h.GetXaxis().GetXmin()+0.05*(h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())
starty10=1.0*h.GetMaximum()
starty=0.9*h.GetMaximum()
starty1=0.8*h.GetMaximum()
starty30=0.7*h.GetMaximum()
starty2=0.6*h.GetMaximum()
starty3=0.5*h.GetMaximum()
starty0=0.4*h.GetMaximum()

summ= ROOT.TLatex(startx, starty, f"#mu1:{fitFcn.GetParameter(1):5.4f}#pm{fitFcn.GetParError(1):5.4f}")
sum1= ROOT.TLatex(startx, starty1, f"#sigma1:{abs(fitFcn.GetParameter(2)):5.4f}#pm{fitFcn.GetParError(2):5.4f}")
sum10= ROOT.TLatex(startx, starty10, f"Yield1:{abs(Yield1):.0f}#pm{err_xi1320_yield:.0f}")
sum2= ROOT.TLatex(startx, starty2, f"mu2:{fitFcn.GetParameter(4):5.4f}#pm{fitFcn.GetParError(4):5.4f}")
sum3= ROOT.TLatex(startx, starty3, f"#sigma2:{fitFcn.GetParameter(5):5.4f}#pm{fitFcn.GetParError(5):5.4f}")
sum30= ROOT.TLatex(startx, starty30, f"Yield2:{Yield2:.0f}#pm{err_xi1530_yield:.0f}")
sum4= ROOT.TLatex(startx, starty0, f"#chi2/Ndf:{chi2perndf:5.2f}")
#"""

summ.SetTextSize(0.05)#0.065
summ.SetTextColor(ROOT.kBlue)
summ.Draw("same")
#"""
sum1.SetTextSize(0.05)
sum1.SetTextColor(ROOT.kBlue)
sum1.Draw("same")
sum10.SetTextSize(0.05)
sum10.SetTextColor(ROOT.kBlue)
sum10.Draw("same")
sum2.SetTextSize(0.05)
sum2.SetTextColor(ROOT.kGreen)
sum2.Draw("same")
sum3.SetTextSize(0.05)
sum3.SetTextColor(ROOT.kGreen)
sum3.Draw("same")
sum30.SetTextSize(0.05)
sum30.SetTextColor(ROOT.kGreen)
sum30.Draw("same")
sum4.SetTextSize(0.05)
sum4.SetTextColor(ROOT.kMagenta)
sum4.Draw("same")

l=ROOT.TLine(1.322, 0, 1.322, 500)
l.SetLineColor(1)
l.SetLineStyle(10)
l.Draw("same")
l2=ROOT.TLine(1.532, 0, 1.532, 500)
l2.SetLineColor(1)
l2.SetLineStyle(10)
l2.Draw("same")

legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
#legend1.SetHeader("fall2018-out-fte","C") # option "C" allows to center the header
#legend1.AddEntry("mc","mc","f")
legend1.AddEntry("data","Data-fall2018-out-fte","pe")#lpe
legend1.AddEntry("fitFcn","Sig. + Bg. Fit","l")
legend1.AddEntry("backFcn", "Bg. Fit","l")
legend1.AddEntry("signalFcnXi1320", "#Xi(1320)","l")
legend1.AddEntry("signalFcnXi1530", "#Xi(1530)","l")
legend1.SetTextFont(72)
legend1.SetTextSize(0.8)
legend1.Draw()

c.Draw()

Fit mean:: 1.33436 Fit sigma:: -0.04820 lcut:: 1.47895 ucut:: 1.18977
 FCN=22.2048 FROM MINOS     STATUS=SUCCESSFUL    367 CALLS        4257 TOTAL
                     EDM=4.84222e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.74375e+01   9.71014e+00  -8.20007e-03   3.44156e-05
   2  p1           1.33436e+00   5.05537e-03  -5.58943e-06  -6.90752e-02
   3  p2          -4.81981e-02   7.07295e-03  -1.73881e-06  -3.24657e-02
   4  p3           6.59818e+01   1.28348e+01   5.97432e-02  -2.11514e-05
   5  p4           1.56174e+00   6.71039e-03   1.28522e-06   2.22715e-02
   6  p5           3.19112e-02   8.09779e-03  -8.52442e-06  -3.83934e-02
   7  p6           1.15265e+03   2.56809e+02   3.89359e-02   2.00410e-04
   8  p7          -1.92057e+03   3.82186e+02  -3.06005e-02   2.30697e-04
   9  p8           8.40929e+02   1.38409e+02 

In [34]:
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")

#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 100, 1.2, 1.7), "mm_fkpskp")
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

#hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fte.root").Histo1D(("mc", "mc", 250, 0.5, 2.5), "mm_fkpskp")
hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fall2018_out_fde_test.root").Histo1D(("mc", "mc", 250, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fte.root").Filter("1.21<=mm_fkpskp && mm_fkpskp<=1.5").Histo1D(("mc", "mc", 250, 0.5, 2.5), "mm_fkpskp")
#hmc.Scale(1/hmc.Integral())
hmc.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
hmc.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
hmc.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
hmc.GetXaxis().SetTitleSize(0.05)
hmc.GetYaxis().SetTitleSize(0.05)
hmc.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
hmc.Draw("SAME")
#h1.SetStats(0)
#h.Fit("gaus", "S", " ", 1.31, 1.36)

legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend1.SetHeader("MC Vs data -xi(1320)","C") # option "C" allows to center the header
legend1.AddEntry("mc","mc","f")
legend1.AddEntry("data","data-fall2018-out-fte","lpe")#lpe
legend1.Draw()

c.Draw()

In [35]:
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte_v1.root").Histo1D(("data", "data", 250, 0.5, 3.0), "mm_fkpskp")
h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte_v1.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")

#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte_v1.root").Histo1D(("data", "data", 100, 1.2, 1.7), "mm_fkpskp")
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fte.root").Histo1D(("mc", "mc", 250, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fte.root").Filter("1.21<=mm_fkpskp && mm_fkpskp<=1.5").Histo1D(("mc", "mc", 250, 0.5, 3.0), "mm_fkpskp")
#hmc.Scale(1/hmc.Integral())
hmc.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
hmc.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
hmc.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
hmc.GetXaxis().SetTitleSize(0.05)
hmc.GetYaxis().SetTitleSize(0.05)
hmc.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

%jsroot on
ROOT.gStyle.SetOptStat(0000)
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
hmc.Draw("SAME")
#h1.SetStats(0)
#h.Fit("gaus", "S", " ", 1.31, 1.36)
legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend1.SetHeader("MC Vs data -xi(1320)","C") # option "C" allows to center the header
legend1.AddEntry("mc","mc","f")
legend1.AddEntry("data","data-fall2018-out-fte-addedcorn","f")#lpe
legend1.Draw()
c.Draw()

**RGK-FTe**

In [36]:
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_rgk_out_fte.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","data_xi1320_2kplus_rgk_out_fte.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_rgk_out_fte.root").Histo1D(("data", "data", 100, 1.2, 1.7), "mm_fkpskp")
h.Scale(1/h.Integral())
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rgk_fte.root").Histo1D(("mc", "mc", 250, 0.5, 2.5), "mm_fkpskp")
#hmc = ROOT.RDataFrame("T","mc_rec_xi1320_rga_fte.root").Filter("1.21<=mm_fkpskp && mm_fkpskp<=1.5").Histo1D(("mc", "mc", 250, 0.5, 2.5), "mm_fkpskp")
hmc.Scale(1/hmc.Integral())
hmc.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
hmc.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
hmc.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
hmc.GetXaxis().SetTitleSize(0.05)
hmc.GetYaxis().SetTitleSize(0.05)
hmc.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

%jsroot on
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
#hmc.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
h.Draw("SAME")
hmc.SetLineColor(ROOT.kBlue)
hmc.SetMarkerStyle(25)
hmc.SetMarkerSize(0.8)
#h1.SetStats(0)
#h.Fit("gaus", "S", " ", 1.31, 1.36)
legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
#legend1.SetHeader("MC Vs data -xi(1320)","C") # option "C" allows to center the header
#legend1.AddEntry("mc","mc","lpe")
legend1.AddEntry("data","rgk-7546MeV-fte","lpe")#lpe
legend1.Draw()
c.Draw()

**RGK-7546MeV-FDe**

In [37]:
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","data_xi1320_2kplus_rgk_7546MeV_out_fde.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")

#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 100, 1.2, 1.7), "mm_fkpskp")
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

%jsroot on
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
#h1.SetStats(0)
#h.Fit("gaus", "S", " ", 1.31, 1.36)
legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
#legend1.SetHeader("MC Vs data -xi(1320)","C") # option "C" allows to center the header
#legend1.AddEntry("mc","mc","f")
legend1.AddEntry("data","rgk-7546MeV-fde","lpe")#lpe
legend1.Draw()
c.Draw()

**RGK-6536MeV-FDe**

In [38]:
#h = ROOT.RDataFrame("T","data_xi1320_2kplus_fall2018_out_fte.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")
h = ROOT.RDataFrame("T","data_xi1320_2kplus_rgk_6535MeV_out_fde.root").Histo1D(("data", "data", 250, 0.5, 2.5), "mm_fkpskp")

#h = ROOT.RDataFrame("T","data_xi1320_2kplus_rgk_6535MeV_out_fde.root").Histo1D(("data", "data", 100, 1.2, 1.7), "mm_fkpskp")
h.SetTitle("MM(eK^{+}K^{+})")
#h.SetTitleSize(0.06, "x")
h.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h.GetYaxis().SetTitle("Counts/8 MeV/c^{2}")
h.GetXaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleSize(0.05)
h.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

%jsroot on
ROOT.gStyle.SetOptFit(0000)
c = ROOT.TCanvas()
h.Draw("E")
h.SetLineColor(ROOT.kRed)
h.SetMarkerStyle(21)
h.SetMarkerSize(0.8)
#h1.SetStats(0)
#h.Fit("gaus", "S", " ", 1.31, 1.36)
legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
#legend1.SetHeader("MC Vs data -xi(1320)","C") # option "C" allows to center the header
#legend1.AddEntry("mc","mc","f")
legend1.AddEntry("data","rgk-6535MeV-fde","lpe")#lpe
legend1.Draw()
c.Draw()